In [2]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings,ChatHuggingFace,HuggingFaceEndpoint

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

In [7]:
from langgraph.graph import StateGraph,END
from typing import List,TypedDict

In [8]:
class RAGState(TypedDict):
    question:str
    context:List[Document]
    answer:str

In [9]:
url="https://en.wikipedia.org/wiki/Machine_learning"
web_loader=WebBaseLoader(url)
web_doc=web_loader.load()

In [10]:
## create the text splitter-later for chunking:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n","\n"," ",""],
    length_function=len)

text_splitter

In [12]:
## create the chunks:
chunks=text_splitter.split_documents(web_doc)

In [13]:
## load the embedding model:
model_name="sentence-transformers/all-MiniLM-L6-v2"

embedding_model=HuggingFaceEmbeddings(model_name=model_name)
embedding_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
vector_store=Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model)

vector_store

In [15]:
## create the retriever:
retriever=vector_store.as_retriever(
    search_type="similarity",
    search_kwags={"k":3}
)

retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000229330AB230>, search_kwargs={})

In [16]:
## create the prompt:
prompt=ChatPromptTemplate.from_template(
    """
    You are a useful assistant who answers questions with honesty and integrity.
    Answer the questions using the below context and if the question is not within the context then say
    I don't know
    """
)

In [17]:
def retrieve_node(state:RAGState)->RAGState:
    docs=retriever.invoke(state["question"])

    return {
    "question":state["question"],
    "context":docs
    }

In [ ]:
llm_endpoint=HuggingFaceEndpoint(
    repo_id="",
    task="text-generation",
    max_new_tokens=512,
    do_smaple=True)

chat_model=ChatHuggingFace(llm=llm_endpoint)

In [ ]:
def generation_node(state:RAGState)->RAGState:
    response=(
        prompt
        |chat_model
        |StrOutputParser()
    ).invoke(
        {"question":state["question"],
         "context":state["context"]
        }
    )

    return {"answer":response}

In [ ]:
graph=StateGraph(RAGState)


graph.add_node("retrieve",retrieve_node)
graph.add_node("generation",generation_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve","generation")
graph.add_edge("generation",END)

In [ ]:
user_query=input("Enter question here about machine learning
rag_app=graph.compile()
